# Project: 波士顿房价线性回归

- 目的: 建立和评估用于线性回归问题(波士顿房价)神经网络模型
- 过程:
    - 加载数据
    - 正则化数据(标准化)
    - 建立神经网络模型
    - 交叉验证
    - 调节网络参数

项目内容主要包括:

1. 使用原始数据训练一个基准神经网络模型(13-13-1),用交叉验证评估模型, 并作为基准.
2. 正则化(Standardize)原始数据, 并训练该基准神经网络模型, 交叉验证评估模型;
3. 调整神经网络模型拓扑结构为(13-13-6-1), 使用正则化数据训练该基准神经网络模型, 交叉验证评估模型;
4. 调整神经网络模型隐含层节点数目(13-20-1), 使用正则化数据训练该基准神经网络模型, 交叉验证评估模型.

让我们开始吧.

## 1. 波士顿房价数据集

- 数据来源: [housing_data](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data)
- 数据说明: [housing_name](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names)
- 数据信息: 
    - 主要关注: 关注波士顿郊区的房价。
    - 数据样本数目: 506个
    - 样本特征: 13个
```
        1. CRIM      per capita crime rate by town
        2. ZN        proportion of residential land zoned for lots over 25,000 sq.ft.
        3. INDUS     proportion of non-retail business acres per town
        4. CHAS      Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        5. NOX       nitric oxides concentration (parts per 10 million)
        6. RM        average number of rooms per dwelling
        7. AGE       proportion of owner-occupied units built prior to 1940
        8. DIS       weighted distances to five Boston employment centres
        9. RAD       index of accessibility to radial highways
        10. TAX      full-value property-tax rate per $10,000
        11. PTRATIO  pupil-teacher ratio by town
        12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        13. LSTAT    % lower status of the population
```

    - 样本房价(MEDV): 1个
        - MEDV: Median value of owner-occupied homes in $1000's

In [1]:
import pandas as pd
# 数据加载
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
df = pd.read_csv(url, delim_whitespace=True, header=None)

In [2]:
df.shape

(506, 14)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [4]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0,0.573,6.030,80.8,2.5050,1,273.0,21.0,396.90,7.88,11.9


## 2.建立神经网络模型基准

使用原始数据训练一个基准神经网络模型(13-13-1),用交叉验证评估模型, 并作为基准.

In [5]:
# 基准神经网络模型
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 数据加载
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
df = pd.read_csv(url, delim_whitespace=True, header=None)
dataset = df.values
# 样本特征和标签划分
X = dataset[:, 0:13]
Y = dataset[:, 13]

# 基准NN模型
def baseline_model():
    # 创建模型
    model = Sequential()
    model.add(Dense(13, input_dim=13, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # 编译模型
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# 随机数参数
seed = 7
np.random.seed(seed)
# 评估模型
reg = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(reg, X, Y, cv=kfold)
print("基准NN(MSE-均方根值): {0:.2f} ({1:.2f})".format(results.mean(), results.std()))

Using Theano backend.
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(13, activation="relu", kernel_initializer="normal", input_dim=13)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`


基准NN(MSE-均方根值): 55.88 (40.94)


## 3.标准化数据训练基准NN模型

正则化(Standardize)原始数据, 并训练该基准神经网络模型, 交叉验证评估模型;

In [6]:
# 正则化数据集训练神经网络模型

# 随机数参数
seed = 7
np.random.seed(seed)
# 正则化数据训练, 评估模型
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, nb_epoch=50, 
                                       batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold= KFold(n_splits=10, random_state=seed)
results_stand = cross_val_score(pipeline, X, Y, cv=kfold)
print("标准化数据NN模型(MSE): {0:.2f} ({1:.2f})".format(results_stand.mean(),
                                                results_stand.std()))

D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(13, activation="relu", kernel_initializer="normal", input_dim=13)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`


标准化数据NN模型(MSE): 51.47 (34.84)


## 4. 调整神经网络拓扑结构

调整神经网络模型拓扑结构为(13-13-6-1), 使用正则化数据训练该基准神经网络模型, 交叉验证评估模型;

In [7]:
# 调整神经网络拓扑结构: 4层 13 - [13 - 6] - 1
# 模型定义
def larger_model():
    # 创建模型
    model = Sequential()
    model.add(Dense(13, input_dim=13, init='normal', activation='relu'))
    model.add(Dense(6, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # 编译模型
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# 随机数
seed = 7
np.random.seed(seed)
# 标准化数据训练模型并评估
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, nb_epoch=50, batch_size=5,
                                        verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results_larger_nn = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger NN MSE: {0:.2f} ({1:.2f})".format(results_larger_nn.mean(), 
                                               results_larger_nn.std()))

D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(13, activation="relu", kernel_initializer="normal", input_dim=13)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="normal")`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`


Larger NN MSE: 37.98 (35.32)


## 5. 调整隐含层节点数

调整神经网络模型隐含层节点数目(13-20-1), 使用正则化数据训练该基准神经网络模型, 交叉验证评估模型.

In [9]:
# 神经网络中间层节点数多的模型: wider = 13-20-1
# 模型定义
def wider_model():
    # 创建模型
    model = Sequential()
    model.add(Dense(20, input_dim=13, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # 编译模型
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# 随机数
seed = 7
np.random.seed(seed)
# 正则化数据训练模型并评估
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, nb_epoch=100,
                                        batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results_wider = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider NN MSE: {0:.2f} ({1:.2f})".format(results_wider.mean(), 
                                               results_wider.std()))

D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, activation="relu", kernel_initializer="normal", input_dim=13)`
D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`


Wider NN MSE: 47.47 (39.22)
